In [18]:
import pandas as pd  # This is always assumed but is included here as an introduction.
import numpy as np
import matplotlib.pyplot as plt

EVERYTHING_AFTER_BACKSLASH = '(?<=\\\\).+' # regex for anything after a backslash

df = pd.read_csv('../data/years/2020-21/2020-21_Player_PerGame.csv')

# Add column [PlayerID]
df['PlayerID'] = df['Player'].str.extract('(' + EVERYTHING_AFTER_BACKSLASH + ')')

# [Player] cleaning -- remove backslash + player ID
df = df.replace(EVERYTHING_AFTER_BACKSLASH, '', regex=True)
df = df.replace('\\\\', '', regex=True)

# [Rk] drop -- unnecessary
df = df.drop('Rk', 1)

df['Pos'].value_counts()

SG       162
PF       143
C        138
PG       127
SF       119
SF-PF      4
SF-SG      3
SG-PG      2
C-PF       2
SG-SF      2
PF-C       1
PG-SG      1
PF-SF      1
Name: Pos, dtype: int64

In [21]:
df.loc[df['Pos'] == 'SF-PF']

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PlayerID
124,Gary Clark,SF-PF,26,TOT,39,11,16.8,1.0,3.4,0.303,...,0.8,2.1,2.9,0.8,0.3,0.2,0.4,0.9,3.1,clarkga01
138,Tyler Cook,SF-PF,23,TOT,32,1,13.7,2.2,3.2,0.670,...,0.9,2.1,3.0,0.5,0.3,0.1,0.7,1.1,4.9,cookty01
373,Rodions Kurucs,SF-PF,22,TOT,21,0,6.0,0.5,1.5,0.344,...,0.2,0.9,1.1,0.5,0.4,0.2,0.6,0.8,1.5,kurucro01
528,Otto Porter Jr.,SF-PF,27,TOT,28,6,21.6,3.5,8.1,0.432,...,1.0,4.4,5.4,2.0,0.6,0.1,0.9,1.5,9.7,porteot01


In [25]:
df.loc[df['Player'] == 'Otto Porter Jr.']

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PlayerID
528,Otto Porter Jr.,SF-PF,27,TOT,28,6,21.6,3.5,8.1,0.432,...,1.0,4.4,5.4,2.0,0.6,0.1,0.9,1.5,9.7,porteot01
529,Otto Porter Jr.,SF,27,CHI,25,6,21.6,3.6,8.1,0.441,...,1.1,4.4,5.5,2.0,0.5,0.2,1.0,1.5,9.9,porteot01
530,Otto Porter Jr.,PF,27,ORL,3,0,22.0,3.0,8.3,0.360,...,0.0,4.7,4.7,1.7,1.3,0.0,0.0,1.3,8.0,porteot01
